In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import json
import time
import openai


In [1]:
!pip freeze

absl-py==0.13.0
-e git+https://github.com/yz-joey/ACLPUB.git@b8df276e8f2e14e96a424a025ef010aed826d79c#egg=aclpub_check
adal==1.2.7
aiohttp==3.8.1
aiosignal==1.2.0
annotated-types==0.7.0
ansiwrap==0.8.4
anyio==4.6.0
appnope==0.1.2
argon2-cffi==20.1.0
astunparse==1.6.3
async-generator==1.10
async-timeout==4.0.2
attrs==21.2.0
azure-core==1.26.1
Babel==2.9.1
backcall==0.2.0
backports.entry-points-selectable==1.1.0
bertopic==0.16.3
bleach==3.3.0
blis==0.7.9
Brotli==1.0.9
cachetools==4.2.2
catalogue==2.0.8
certifi==2021.5.30
cffi==1.14.5
chardet==4.0.0
charset-normalizer==2.1.0
clang==5.0
click==7.1.2
codecov==2.1.13
comm==0.2.2
confection==0.0.4
coverage==7.5.3
crowdtruth==2.1
cryptography==38.0.3
cycler==0.10.0
cymem==2.0.7
datasets==2.3.2
dateparser==1.2.0
decorator==4.4.2
defusedxml==0.7.1
dill==0.3.4
distlib==0.3.2
distro==1.9.0
entrypoints==0.3
exceptiongroup==1.0.4
filelock==3.0.12
fire==0.4.0
Flask==1.1.2
flatbuffers==1.12
frozenlist==1.3.0
fsspec==2024.3.1
gast==0.4.0
gensim==3.8.3


### Open AI: setting key

In [2]:
import openai
from openai import OpenAI

openai.api_key = key

NameError: name 'key' is not defined

In [3]:
GPT_4o = "gpt-4-turbo-2024-04-09"
GPT_3 = "gpt-3.5-turbo-0125"

In [4]:
from openai.types import Completion, CompletionChoice, CompletionUsage

In [5]:
client = OpenAI(api_key=key)

In [8]:
completion = client.chat.completions.create(model=GPT_4o, messages=[{"role": "user", "content": "Hello world!"}])

In [9]:
## test
completion.choices[0].message.content

'Hello! How can I assist you today?'

## Prompt Section

In [11]:
## original prompt
def create_prompt_definition_sexism(definition):

    prompt = f"Your task is to annotate if a piece of text is sexist or not.\
    Later, you get a detailed definition of sexism. \
    Definition of sexism: {definition} "
    
    return prompt

In [12]:
test_text1 = "women are capable of commentating on football matches" 
test_text2 = "women are weak and belong in the kitchen"

test_definition = "sexism is when men comment on women"

In [13]:
prompt = create_prompt_definition_sexism(test_definition)

In [14]:
prompt

'Your task is to annotate if a piece of text is sexist or not.    Later, you get a detailed definition of sexism.     Definition of sexism: sexism is when men comment on women '

#### import definitions, put in dictionary per participant

In [15]:
definitions_data = pd.read_csv("../../data/definitions/Sexism/Definitions_Interaction_ds - Definitions_Interaction_ds.csv")

In [16]:
from collections import defaultdict

def_dict = defaultdict()

for _, row in definitions_data.iterrows():
    participant_dict = defaultdict()
    participant_dict["chosen_definition_10"] = row["chosen_definition_10"]
    participant_dict["def_ChatGPT_1"] = row["def_ChatGPT_1"]
    participant_dict["participant_definition"] = row["participant_define"]
    try: 
        participant_dict["definition_category"] = row["Indira Definition Category"].split(" detail")[0]
    except:
        participant_dict["definition_category"] = "unk"
    def_dict[row["ResponseId"]] = participant_dict
    
    

In [18]:
# def_dict

#### add definition prompts to dict

In [19]:
for participant in def_dict.keys():
    def_dict[participant]["prompt_definition_chosen_10"] = create_prompt_definition_sexism(def_dict[participant]["chosen_definition_10"])
    def_dict[participant]["prompt_definition_ChatGPT_1"] = create_prompt_definition_sexism(def_dict[participant]["def_ChatGPT_1"])
    def_dict[participant]["prompt_definition_participant"] = create_prompt_definition_sexism(def_dict[participant]["participant_definition"])


In [22]:
# def_dict

#### Add texts to prompt

In [23]:
def add_texts_to_prompts(prompt_dict, prompt_version, text):
    preamble = prompt_dict[prompt_version]
    prompt = preamble + f"With this definition in mind and a piece of text, please reply with either of the three options and nothing else: 1) sexist, 2) non-sexist, 3) can’t say. Please use the last option as little as possible and if you do, provide a one sentence explanation. Do not give an explanation if you pick ’sexist’ or ’non-sexist’. \
    We are ready to begin now. Is this sentence sexist based on the past definition’ : {text} - ’Please reply with either of the three options and nothing else: 1) sexist, 2) non-sexist, 3) can’t say."
    return prompt

In [103]:
# def_dict

### Classification test 

In [24]:
prompt = add_texts_to_prompts(participant_dict, 
                             "prompt_definition_participant", 
                            "Girls are dumber than boys")

completion = client.chat.completions.create(model=GPT_4o, 
                                            messages=[{"role": "user", 
                                                       "content": prompt}],n=3)

In [25]:
completion.choices[0].message.content

'1) sexist'

### Test prompts with definitions on our sexism datasets

In [26]:
test_sets = ['in_domain', 'out_of_domain', 
             'out_of_domain_2', 'out_of_domain_3',# 'out_of_domain_4', 
             'hatecheck'
]

In [27]:
construct_names = ['sexism']

construct = 'sexism'
pos_label = 'sexist'
neg_label = 'non-sexist'

In [28]:
test_set_data = {}

for construct in construct_names:
    test_set_data[construct] = {}
    for test_set in test_sets:
        test_path = '../../data/data/%s/test/%s.csv' %(construct, test_set)
        test_set_data[construct][test_set] = pd.read_csv(test_path, sep = '\t').sample(n=500, random_state=123)

In [300]:
#  test_set_data["sexism"]

In [29]:
sample_data = pd.read_csv("../../data/data/sexism/train/data_sample.tsv",  sep = '\t')

In [30]:
sample_data[:5]

,_id,text,sexism
0,5d654925ce2c6bd7334795bd,i can't take girls serious when talk about sports,sexist
1,5d654901ce2c6bd733475c03,#womensday Women have surely outdone men if t...,sexist
2,5d65490ace2c6bd7334763c0,You've undercooked eggs and probably poisoned ...,non-sexist
3,5ddc4fe8174b03a4c6aaf5d5,A daughter in a family should have the same pr...,non-sexist
4,5d654921ce2c6bd733478ac5,a womans place IS the kitchen,sexist


In [39]:
def_dict.keys()

dict_keys(['EXP_1', 'EXP_2', 'EXP_3', 'EXP_4', 'EXP_5', 'EXP_6', 'EXP_7', 'EXP_8', 'EXP_9', 'EXP_10', 'EXP_11'])

### clean_def_dict with empty definitions (participants didn't do def experiment)

In [43]:
#clean_def_dict with empty definitions (participants didn't do def experiment)
participants_to_remove = ('EXP_7', 'EXP_8')
for k in participants_to_remove:
    def_dict.pop(k, None)

In [44]:
def_dict.keys()

dict_keys(['EXP_1', 'EXP_2', 'EXP_3', 'EXP_4', 'EXP_5', 'EXP_6', 'EXP_9', 'EXP_10', 'EXP_11'])

In [3]:
# test_set_data["sexism"].items()

## Classification w open AI models

#### We run the following loop for all texts for each of the three definition dictionaries: (1) for the participant definitions, (2) for the GPT definitions, and (3) for the co-created definitions.

### Loop 1: participant definitions

In [47]:
results_participantdef_GPT4o = defaultdict()

In [48]:
results_participantdef_GPT4o

defaultdict(None, {})

In [120]:
# for each test set
# for domain, test_set in test_set_data["sexism"].items():

for data in ["hatecheck", "out_of_domain_2", "out_of_domain_3"]:
    domain = data
    test_set = test_set_data["sexism"][domain]
    # we have a result dictionary of dictionaries per participant
    result_dict = defaultdict()
    for participant, participant_dict in def_dict.items():
            part_dict = defaultdict()
            for _, row in test_set_data["sexism"][domain].iterrows():
                prompt = add_texts_to_prompts(participant_dict, 
                                          "prompt_definition_participant", 
                                          row["text"])
                try:
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        n=2,
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0)                                                                    
                except:
                    time.sleep(60)
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                         n = 2, 
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0) 
                part_dict[row["_id"]] = [responses.choices[0].message.content,
                                         responses.choices[1].message.content]
            results_participantdef_GPT4o[participant] = part_dict 
            result_dict['GPT4o'] = results_participantdef_GPT4o
    test_set_data['sexism'][domain]["results_stabilityCheck"] = str(result_dict)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### writing the dictionary of results 

In [415]:
with open("GESIS_stability_n2.txt", "w+") as f:
    f.write(str(result_dict))

In [116]:
import pickle

with open('all_results_dict_of_dfs_stability_n2_GESIS.pickle', 'wb') as f:
    pickle.dump(test_set_data, f)

### Loop 2: GPT definitions

In [ ]:
# for each test set
for domain, test_set in test_set_data["sexism"].items():
    # we have a result dictionary of dictionaries per participant
    result_dict = defaultdict()
    for participant, participant_dict in def_dict.items():
            part_dict = defaultdict()
            for _, row in test_set_data["sexism"][domain].iterrows():
                prompt = add_texts_to_prompts(participant_dict, 
                                          "prompt_definition_ChatGPT_1", 
                                          row["text"])
                try:
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0)                                                                    
                except:
                    time.sleep(60)
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0) 
                part_dict[row["_id"]] = responses.choices[0].message.content                                                          
            results_participantdef_GPT4o[participant] = part_dict 
            result_dict[domain] = results_participantdef_GPT4o
    test_set_data['sexism'][domain]["results_GPTdef1"] = str(result_dict)

In [ ]:
with open("F1_500items_GPTdef1.txt", "w+") as f:
    f.write(result_dict)

### Loop 3: co-created definitions

In [750]:
# for each test set
for domain, test_set in test_set_data["sexism"].items():
    # we have a result dictionary of dictionaries per participant
    result_dict = defaultdict()
    for participant, participant_dict in def_dict.items():
            part_dict = defaultdict()
            for _, row in test_set_data["sexism"][domain].iterrows():
                prompt = add_texts_to_prompts(participant_dict, 
                                          "prompt_definition_chosen_10", 
                                          row["text"])
                try:
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0)                                                                    
                except:
                    time.sleep(60)
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0) 
                part_dict[row["_id"]] = responses.choices[0].message.content                                                          
            results_participantdef_GPT4o[participant] = part_dict 
            result_dict[domain] = results_participantdef_GPT4o
    test_set_data['sexism'][domain]["results_GPTdef10"] = str(result_dict)

In [1]:
# test_set_data["sexism"]["in_domain"][:5]

In [752]:
with open("F1_500items_GPTdef10.txt", "w+") as f:
    f.write(str(result_dict))

In [753]:
import pickle

with open('Loop3_results_dict_of_dfs.pickle', 'wb') as f:
    pickle.dump(test_set_data, f)

In [16]:
results_all_test = pd.read_pickle("Loop3_results_dict_of_dfs.pickle")

### Make metrics dataframe for GPT definitions

In [371]:
metrics_df = pd.DataFrame()

definitions = ["results_GPTdef10", "results_GPTdef1"]
experts = ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9", "EXP_10", "EXP_11"]

for dataset in results_all["sexism"].keys():
    for definition in definitions:
        for expert in experts: 
            for i in results_all["sexism"][dataset][definition][:2]:
                result_dict = eval(i)
#                 print(result_dict)
                try:
                    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict[dataset][expert].values()]
                    labels = ["non-sexist" if i == "can’t say" or i == "can't say" else i for i in labels] 
                    truth = results_all["sexism"][dataset]["sexism"].tolist()
                    scores = classification_report(truth, labels, output_dict=True)

                    temp_dict = pd.DataFrame()

                    temp_dict["definition"] = [definition]
                    temp_dict["expert"] = expert
                    temp_dict["dataset"] = dataset
                    temp_dict["scores"] = [scores]

                    metrics_df = pd.concat([metrics_df, temp_dict])
                except:
                    print("skip")

skip
skip


### Add the participant definitions to this dataframe

In [397]:
for dataset in results_all["sexism"].keys():
    for definition in ["results"]:
        for expert in experts: 
            for i in results_all["sexism"][dataset][definition][:2]:
                result_dict = eval(i)
#                 print(result_dict)
                try:
                    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["GPT4o"][expert].values()]
                    labels = ["non-sexist" if i == "can’t say" or i == "can't say" else i for i in labels] 
                    truth = results_all["sexism"][dataset]["sexism"].tolist()
                    scores = classification_report(truth, labels, output_dict=True)

                    temp_dict = pd.DataFrame()

                    temp_dict["definition"] = [definition]
                    temp_dict["expert"] = expert
                    temp_dict["dataset"] = dataset
                    temp_dict["scores"] = [scores]

                    metrics_df = pd.concat([metrics_df, temp_dict])
                except:
                    print("skip")

In [398]:
metrics_df["definition"] = metrics_df["definition"].replace("results_GPTdef10", "hybrid")
metrics_df["definition"] = metrics_df["definition"].replace("results_GPTdef1", "GPT")
metrics_df["definition"] = metrics_df["definition"].replace("results", "participant")

In [200]:
for i in metrics_df["scores"][:1]:
    print(i)

{'non-sexist': {'precision': 0.9333333333333333, 'recall': 0.7608695652173914, 'f1-score': 0.8383233532934132, 'support': 276}, 'sexist': {'precision': 0.76, 'recall': 0.9330357142857143, 'f1-score': 0.8376753507014029, 'support': 224}, 'accuracy': 0.838, 'macro avg': {'precision': 0.8466666666666667, 'recall': 0.8469526397515528, 'f1-score': 0.837999351997408, 'support': 500}, 'weighted avg': {'precision': 0.8556800000000001, 'recall': 0.838, 'f1-score': 0.8380330481321926, 'support': 500}}


### add individual metrics to dataframe

In [399]:

F1s = []
in_sex = []
in_non = []

for index, row in metrics_df.iterrows():
    F1 = dict(row["scores"])["macro avg"]["f1-score"]
    class_sexism_f1 = dict(row["scores"])["sexist"]["f1-score"]
    non_sex = dict(row["scores"])["non-sexist"]["f1-score"]
    F1s.append(F1)
    in_sex.append(class_sexism_f1)
    in_non.append(non_sex)
    
#     print(F1)
#     print(in_sex)

metrics_df["F1"] = F1s
metrics_df["sexist_F1"] = in_sex
metrics_df["non_sexist_F1"] = in_non
metrics_df

,definition,expert,dataset,scores,F1,sexist_F1,non_sexist_F1
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.837999,0.837675,0.838323
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.837999,0.837675,0.838323
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0.831957,0.829268,0.834646
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0.831957,0.829268,0.834646
0,hybrid,EXP_3,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.811852,0.817121,0.806584
...,...,...,...,...,...,...,...
0,participant,EXP_9,hatecheck,{'non-sexist': {'precision': 0.267857142857142...,0.483775,0.631415,0.336134
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0.476382,0.661743,0.291022
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0.476382,0.661743,0.291022
0,participant,EXP_11,hatecheck,{'non-sexist': {'precision': 0.271551724137931...,0.484414,0.623622,0.345205


### clean data of duplicates and participants who didn't do it

In [18]:
clean_data = metrics_df[metrics_df[["definition", "expert", "dataset", "F1", "sexist_F1"]].duplicated()]

KeyError: "None of [Index(['definition', 'expert', 'dataset', 'F1', 'sexist_F1'], dtype='object')] are in the [columns]"

In [19]:
clean_data = clean_data[clean_data.expert != "EXP_8"]
clean_data = clean_data[clean_data.expert != "EXP_7"]
clean_data

NameError: name 'clean_data' is not defined